In [4]:
import pandas
# python 환경 : pip install google-api-python-client
# anaconda환경 : conda install -c conda-forge google-api-python-client
from googleapiclient.discovery import build

# 유트브 APi : https://developers.google.com/youtube/v3/getting-started?hl=ko

# 예시 유트브 링크 : https://www.youtube.com/watch?v=tZolEtoQyuY
# 자신 API KEY 사용
api_key = ''

# 비디오 아이디 : tZolEtoQyuY
video_id = 'tZolEtoQyuY'

# 댓글을 담을 변수
comments = list()
# youtebe, v3 api 사용 developerKey는 발급받은 키 하루 10000건 조회가능
api_obj = build('youtube', 'v3', developerKey=api_key)

# https://www.youtube.com/watch?part='snippet,replies'&vieoid='asdasd'&maxResults=100이와 같이 요청을 함 snippet필수속성, replies댓글
response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=10000).execute()

while response:
    for item in response['items']:
        # 분석해야함!
        print(item)
        
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
        if item['snippet']['totalReplyCount'] > 0:
            for reply_item in item['replies']['comments']:
                reply = reply_item['snippet']
                comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
 
    if 'nextPageToken' in response:
        response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
    else:
        break
 

{'kind': 'youtube#commentThread', 'etag': 'okrCL43bUfFbm-Wk8CTxX7B7T_Y', 'id': 'UgxY4VafwSE8WI6LE754AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'QY6z3OqcxvHBKlQj5eIqMt6nrNw', 'id': 'UgxY4VafwSE8WI6LE754AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'textDisplay': '<a href="https://www.youtube.com/watch?v=tZolEtoQyuY&amp;t=01m21s">01:21</a> 맡겨논 내눈', 'textOriginal': '01:21 맡겨논 내눈', 'authorDisplayName': '우름', 'authorProfileImageUrl': 'https://yt3.ggpht.com/O3PuLBkBidC2eZwxuIjFO_nCpSrIQLqTMM1X_CxrpHsfXtWUQHvEum8wrDyxLtCuKZzEgN5i=s48-c-k-c0x00ffffff-no-rj', 'authorChannelUrl': 'http://www.youtube.com/channel/UCypLqBPwD3h_4oOL6Awbliw', 'authorChannelId': {'value': 'UCypLqBPwD3h_4oOL6Awbliw'}, 'canRate': True, 'viewerRating': 'none', 'likeCount': 0, 'publishedAt': '2023-02-03T17:30:47Z', 'updatedAt': '2023-02-03T17:30:47Z'}}, 'canReply': True, 'totalReplyCount': 0, 'isPublic': True}}
{'kind': 'youtube#commentThread', 'etag': 'iVgIetimu

{'kind': 'youtube#commentThread', 'etag': 'LkvDn56d_3gf4b86Jebp-2UeEd8', 'id': 'UgzTH-YvVbqat3dfrR94AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'YpBrXc1ftG-WjWRxFQQHtiG3JEQ', 'id': 'UgzTH-YvVbqat3dfrR94AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'textDisplay': '아니 미원하고 맛소금을 저렇게 넣는다고...?? 나도 10년 자취요리하면서 미원 애용하는데 1인분 요리기준으로 5~10알갱이 넣으면 맛이 확바뀌고 그 이상 넣으면 미원 특유의 맛때문에 역해서 못겠던데...', 'textOriginal': '아니 미원하고 맛소금을 저렇게 넣는다고...?? 나도 10년 자취요리하면서 미원 애용하는데 1인분 요리기준으로 5~10알갱이 넣으면 맛이 확바뀌고 그 이상 넣으면 미원 특유의 맛때문에 역해서 못겠던데...', 'authorDisplayName': 'SeungGi Kim', 'authorProfileImageUrl': 'https://yt3.ggpht.com/nP8FKTPvvH3xCBP3AvC737EjzBBrf4ip-HsSeyLSfm7tnNWxIfPEuSRgcS79SQ5tjMgWTjUA9w=s48-c-k-c0x00ffffff-no-rj', 'authorChannelUrl': 'http://www.youtube.com/channel/UCohmGGBGHqJGc4RK8hdl_zg', 'authorChannelId': {'value': 'UCohmGGBGHqJGc4RK8hdl_zg'}, 'canRate': True, 'viewerRating': 'none', 'likeCount': 0, 'publishedAt': '2022-12-07T17:17:29Z', 'updat

{'kind': 'youtube#commentThread', 'etag': 'OPp8J8rvy5qFTQW9UbqoKQ2JvpY', 'id': 'UgwSpX-rSg6HLl6HmGR4AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'EPM_OXbhJBWj5MV0wLUqoKvjgtQ', 'id': 'UgwSpX-rSg6HLl6HmGR4AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'textDisplay': '처음부터 고춧가루랑 조미료 엄청 넣는다 싶엇는데 조절 못해서 물 계속 붓는거 개웃기네 ㅋㅋㅋㅋㅋ', 'textOriginal': '처음부터 고춧가루랑 조미료 엄청 넣는다 싶엇는데 조절 못해서 물 계속 붓는거 개웃기네 ㅋㅋㅋㅋㅋ', 'authorDisplayName': '주너ᄏ', 'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AL5GRJVb52lFJ9920D5u1-a_X9XtlegrBwIcaa-nrTd96Q=s48-c-k-c0x00ffffff-no-rj', 'authorChannelUrl': 'http://www.youtube.com/channel/UCgPHdThwvycPeWl2Zj0e88w', 'authorChannelId': {'value': 'UCgPHdThwvycPeWl2Zj0e88w'}, 'canRate': True, 'viewerRating': 'none', 'likeCount': 2, 'publishedAt': '2022-12-05T04:28:58Z', 'updatedAt': '2022-12-05T04:28:58Z'}}, 'canReply': True, 'totalReplyCount': 0, 'isPublic': True}}
{'kind': 'youtube#commentThread', 'etag': 'FctcleAt7oWbVihQ8U2

{'kind': 'youtube#commentThread', 'etag': '2M1SH92jtEQta_zoYMdWHl15O9w', 'id': 'UgxvgvIWOIi_CiZUemd4AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'iB2w_dMD-G__a8NcoIQfHln4njA', 'id': 'UgxvgvIWOIi_CiZUemd4AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'textDisplay': '<a href="https://www.youtube.com/watch?v=tZolEtoQyuY&amp;t=5m49s">5:49</a> ???: &quot;얘들아 호민이가 할 말 있대&quot;', 'textOriginal': '5:49 ???: "얘들아 호민이가 할 말 있대"', 'authorDisplayName': '훠이', 'authorProfileImageUrl': 'https://yt3.ggpht.com/92o6tSZPivdHK7UYNn0DCOXSrSXxUDzfMTiJgkD3eEDHfpfGMimDulu0oI5YyFgyuAvynmO89g=s48-c-k-c0x00ffffff-no-rj', 'authorChannelUrl': 'http://www.youtube.com/channel/UCFTiMBxZhNoGbskAb_U4L6w', 'authorChannelId': {'value': 'UCFTiMBxZhNoGbskAb_U4L6w'}, 'canRate': True, 'viewerRating': 'none', 'likeCount': 0, 'publishedAt': '2022-12-04T12:54:10Z', 'updatedAt': '2022-12-04T12:54:10Z'}}, 'canReply': True, 'totalReplyCount': 0, 'isPublic': True}}
{'kind': 'y

{'kind': 'youtube#commentThread', 'etag': 'y_ahDZAf6YLLQwez4lhdTuVQyJc', 'id': 'UgwBHyndvbCM9gdmemp4AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'topLevelComment': {'kind': 'youtube#comment', 'etag': '8Pd8TCieEkWc489DpnG7LmFc20I', 'id': 'UgwBHyndvbCM9gdmemp4AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'textDisplay': '난 4인분에 미원 반스푼도 안 넣음 근데 미원 맛이 나던데 저정도면…..너무 과하네요.', 'textOriginal': '난 4인분에 미원 반스푼도 안 넣음 근데 미원 맛이 나던데 저정도면…..너무 과하네요.', 'authorDisplayName': 'HY LEE', 'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AL5GRJWcRnfpJCwx0RG1tZIv9_cJITe4xS5w_hy9Qw=s48-c-k-c0x00ffffff-no-rj', 'authorChannelUrl': 'http://www.youtube.com/channel/UCBtfhRQwN6ptHEWmrRbxWug', 'authorChannelId': {'value': 'UCBtfhRQwN6ptHEWmrRbxWug'}, 'canRate': True, 'viewerRating': 'none', 'likeCount': 2, 'publishedAt': '2022-12-04T11:08:32Z', 'updatedAt': '2022-12-04T11:09:44Z'}}, 'canReply': True, 'totalReplyCount': 0, 'isPublic': True}}
{'kind': 'youtube#commentThread', 'etag': 'YFXvddRAQ37m4zY-e5CZpOZtQ6Q', 

{'kind': 'youtube#commentThread', 'etag': '-88TzOz1ZiNV6zQlVYS04Y8MWgo', 'id': 'UgwNVu2ovQJvhPnRO_d4AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'BF-1VeI3o9uNLWz0WVFV1h2aAs4', 'id': 'UgwNVu2ovQJvhPnRO_d4AaABAg', 'snippet': {'videoId': 'tZolEtoQyuY', 'textDisplay': '<a href="https://www.youtube.com/watch?v=tZolEtoQyuY&amp;t=20m18s">20:18</a> ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 'textOriginal': '20:18 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 'authorDisplayName': '알수없는계정', 'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AL5GRJW4xpjl_YtWOYLykN92HFQDZied0OiKD8yDcXFM=s48-c-k-c0x00ffffff-no-rj', 'authorChannelUrl': 'http://www.youtube.com/channel/UClVhxVlo6gZw7nHIBwGOKBw', 'authorChannelId': {'value': 'UClVhxVlo6gZw7nHIBwGOKBw'}, 'canRate': True, 'viewerRating': 'none', 'likeCount': 2, 'publishedAt': '2022-12-04T10:23:15Z', 'updatedAt': '2022-12-04T10:23:15Z'}}, 'canReply': True, 'totalReplyCount': 0, 'isPublic': True}}
{'kind': 'youtube#commentThread', 'etag': '2ph7CPBdx

In [ ]:
comments